# A Basic Delve into Python NLP
### *~ According to the Majestic Whims of Jay Kaiser ~*

**Note**: A vast majority of the code and methodologies found here has been taken directly from [Modern NLP in Python](http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb). This is merely a way for me to practice these topics myself in my own environment, using the most recent dataset as provided by Yelp. I do not claim original creation of anything found here, and all credit belongs with the authors of that Jupyter notebook. I am merely striving to learn these concepts myself in a manner that is most conductive to my own research...

With that in mind, let's get started.

## Introduction

Hi. My name is Jay Kaiser, and I am in my final semester of grad school at Indiana University Bloomington, where I am receiving my M.S. in Computational Linguistics. Naturally, this has consisted mostly of classes of linguistics and of computer science separately, but occasionally a class that cleverly combines the two appears. Classes like these have slowly migrated me into the world of syntax and syntactic parsing. Moreover, such classes have taken me as well to the wonderful world of data science and data analytics, and my interest for such was only expanded upon following an internship with Kingfisher Systems, Inc. as a data scientist in Washington D.C. over the summer of 2017.

However, I must continue in my studies if I ever hope to succeed. In order to rapidly acclimate myself to data science and its many aspects, I have invested in a range of resources to assist me with my learning. Nothing, though, will teach me better than merely getting my hands dirty and doing it myself, so here it goes...

### Project: Yelp Dataset

I have downloaded on my computer the entirety of the JSON portion of the [Yelp Dataset](https://www.yelp.com/dataset/challenge), which unzipped into six separate JSON files, each of which is outlined below.

| Dataset File | Description |
|-------------:|:------------|
| business.json | information on each business reviewed on Yelp |
| checkin.json | each business' hours |
| photos.json | photos found on Yelp with their captions |
| review.json | text reviews of each business |
| tip.json | small quotes of advice for future visitors of the business |
| user.json | Yelp reviewers and information describing them |

However, only *business.json* and *review.json* will be used here.

### Step 1: A tour of the dataset

First, the files need to loaded into python.

In [1]:
import os
import codecs

data_directory = os.path.join(r'C:\Users\jayka\Documents', 'Datasets', 'yelp')
business_filepath = os.path.join(data_directory, 'business.json')
review_filepath = os.path.join(data_directory, 'review.json')

with codecs.open(business_filepath, encoding='utf_8') as f:
    first_business_record = f.readline()
    
print(first_business_record)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jayka\\Documents\\Datasets\\yelp\\business.json'

Of these, **text** is the field I will use. However, any one of these fields could be used as well for other types of analysis.

This JSON data is not easily navigable, so it'll have to be converted into a more-usable format.

In [ ]:
import json

restaurant_ids = set()

with codecs.open(business_filepath, encoding='utf_8') as f:
    for business_json in f:
        business = json.loads(business_json)
        
        if u'Restaurants' not in business[u'categories']:
            continue
        
        restaurant_ids.add(business[u'business_id'])
    
    restaurant_ids = frozenset(restaurant_ids)
    
    print('{:,}'.format(len(restaurant_ids)), u'restaurants in the dataset.')

There are 51,613 registered restaurants in the dataset, over twice as many as there had been in 2016.

Separately, I'll need to find the reviews for each of the restaurants, based on their ID. This will be placed in a separate file that's easier to parse.

In [ ]:
intermediate_directory = os.path.join(r'C:\Users\jayka\Documents\Datasets', 'intermediate')
review_text_filepath = os.path.join(intermediate_directory, 'yelp_review_text.txt')

In [ ]:
%%time

review_count = 0

if False: # Once this is run once it does not need to be run again.
    
    with codecs.open(review_text_filepath, 'w', encoding='utf_8') as review_text_file:
        with codecs.open(review_filepath, encoding='utf_8') as review_json_file:
            for review_json in review_json_file:
                review = json.loads(review_json)

                if review[u'business_id'] not in restaurant_ids:
                    continue

                review_text_file.write(review[u'text'].replace('\n', '\\n') + '\n')
                review_count += 1
            
    print(u'Text from {:,} restaurant reviews written to the new text file.'.format(review_count))

else:
    with codecs.open(review_text_filepath, encoding='utf_8') as review_text_file:
        for review_count, line in enumerate(review_text_file):
            pass

    print(u'Text from {:,} restaurant reviews written to the new txt file.'.format(review_count + 1))

This process takes me about 2.5 minutes on my dual-core i5 laptop. Moreover, ther are a total of 2,927,731 reviews found, over three times as many as in 2016. I now have the IDs and review texts in separate files, and real parsing of the text can now begin.

### Step 2: Introduction to text processing with spaCy

spaCy is a means to perform NLP tasks on these reviews just collected. It is both free-to-use and incredibly efficient and fast, much moreso than NLTK. It has been designed not for research, but for commerical and industrial usage. Below, a sample review has been printed. With it, I can get a better idea of what kind of data I'm working with.

In [ ]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

with codecs.open(review_text_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')

print(sample_review)

In [ ]:
%%time
parsed_review = nlp(sample_review)

Parsing this entire thing with state-of-the-art NLP technology only took 23.2 milliseconds on my computer. Super fast!

For listing all of the sentences in the sample review:

In [ ]:
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

For listing all of the named entities:

In [ ]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

For listing each word's POS tag:

(We'll be using *pandas* for this for its easy displaying and further parsing.)

In [ ]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

For lemmas and shape analysis:

In [ ]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
            columns=['token_text', 'token_lemma', 'token_shape'])

As you can see, each lemma is its token, minus inflections or tenses. Also note that all pronouns have been converted to *-PRON-*. (I can only guess this is because all pronouns can be see as identical information-wise, so there's no reason to save them as separate lemmas.)

For listing tokens and their analytics (like what kind of named entity they are and where a token lies within a named entity phrase):

In [ ]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
            columns=['token_text', 'entity_type', 'inside_outside_begin'])

For listing stopwords, punctuation, whitespace, numbers, unstandard vocab words, probability of appearance in the corpus, etc:

In [ ]:
token_attributes = [(token.orth_,
                    token.prob,
                    token.is_stop,
                    token.is_punct,
                    token.is_space,
                    token.like_num,
                    token.is_oov)
                   for token in parsed_review]

df = pd.DataFrame(token_attributes,
                 columns=['text',
                         'log_probability',
                         'stop?',
                         'punctuation?',
                         'whitespace?',
                         'number?',
                         'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                      .applymap(lambda x: u'Yes' if x else u''))

df

### Step 3: Automatic Phrase Modeling

The problem with these tokens is that independently, they do not properly reflect the relationships between one another. For example, above, *do* and *n't* are considered separate tokens, but they would be better reflected as a single collocation since *n't* will never appear apart from *do* (or alternative modal verbs).

From the *gensim* library, I will be able to complete phrase modeling over the current list of tokens to find such relationships.

In [ ]:
import warnings
# Because of Windows Anaconda, a warning appears without this filter.
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

There are four steps that must be done in data preparation here:
- a) Segment text of complete reviews into sentences and normalize text
- b) First-order phrase modeling
- c) Second-order phrase modeling
- d) Apply text normalization and second-order phrase model to text of complete reviews

#### a) Segment text of complete reviews into sentences & normalize text

Three helper functions will be used for this step. They will, respectively, find punctuation and whitespace, line breaks, and word lemmas.

In [ ]:
def punct_space(token):
    """ eliminates purely punctuation and whitespace tokens"""
    return token.is_punct or token.is_space

def line_review(filename):
    """reads in each review and unescapes the line-breaks"""
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """uses spaCy to parse reviews, lemmatize, and tokenize sentences"""
    for parsed_review in nlp.pipe(line_review(filename),
                                 batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                            if not punct_space(token)])

*lemmatized_sentence_corpus* loops over the original review text and outputs it to *unigram_sentences_all* one normalized sentence at a time, with one per line.

In [ ]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'unigram_sentences_all.txt')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_text_filepath):
            f.write(sentence + '\n')

This process took me 2 hours and 48 minutes to complete on my modest computer.

The Gensim library provides a LineSentence class that streams documents of this format from disk, allowing massive scaling for huge amounts of data that now does not need to be saved in whole into memory. 

In [ ]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

What do these sentences look like now?

In [ ]:
for unigram_sentence in it.islice(unigram_sentences, 450, 460):
    print(u' '.join(unigram_sentence))
    print(u'')

Notice that these new sentences are far simpler grammatically than they would have been before. This presents a glimpse at what the final outcome of this stage will look like in the end.

#### b) First-order phrase modeling

From these normalized sentences, one can now use *gensim.models.Phrases* to find two-word collocations and link them together into single tokens. 

For example, in one of the sentences above are the words "soy sauce". Though they are technically separate words, when together as a bigram they act as a distinct entity. Hence, by training a model on frequent bigrams, one can disambiguate cases like these.

In [ ]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model_all')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_model_filepath)

# load the finished model back in after it's been built
bigram_model = Phrases.load(bigram_model_filepath)

This process took a little more than 14 minutes to complete on my computer. By saving the model to its filepath, it can be run on unrelated text. This process allows different models formed from different corpora to be cross-compared.

I can now edit the original sentences to account for these bigrams.

In [ ]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                        'bigram_sentences_all.txt')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

This took a little more than 26 minutes on my computer. Let's take a look at what I'm left with.

In [ ]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

for bigram_sentence in it.islice(bigram_sentences, 230, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

So what has been achieved? Looking at the outputted sentences above, now two-word phrases like "deep fried" and "pot stickers" are now written out as "deep\_fried" and "pot\_stickers". This gives a better representation of what words co-occur together at a rate far greater than chance.

#### c) Second-order phrase modeling

We now have concatenated bigrams, but having trigrams accounted for as well might be a good idea. This involves nearly the same exact code found in part (b), but this time instead of running the new trigram model over the unigram sentences, I'll run it over the just-made bigram ones.

In [ ]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                     'trigram_model_all')

In [ ]:
%%time
if False:  # Once this is run once it does not need to be run again.
    
    trigram_model = Phrases(bigram_sentences)
    trigram_model.save(trigram_model_filepath)

# load the finished model back in after it's been built
trigram_model = Phrases.load(trigram_model_filepath)

This process took almost 18 minutes to complete on my computer. Again, I'm saving this trigram model in its own file so that it could be used again in the future on other corpora. Now, this second-order phrase model can be applied to the first-order transformed sentences, and the results of such will be outputted to a third sentences file.

In [ ]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'trigram_sentences_all.txt')

In [ ]:
%%time
if False:  # Once this is run once it does not need to be run again.
    
    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for bigram_sentence in bigram_sentences:
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

This took almost 27 minutes on my computer. Now there is a file consisting of every sentence marked with trigrams. 

In [ ]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [ ]:
for trigram_sentence in it.islice(trigram_sentences, 270, 280):
    print(u' '.join(trigram_sentence))
    print(u'')

Why did I do all this? Because now, instead of the phrase "p.f. chang 's" being represented as three separate words, it is now concatenated into "p.f.\_chang\_'s". This is now correctly marked as a single entity instead of as three.

Technically, I could repeat this process indefinitely with higher and higher ngrams, but very little is gained information-wise after trigrams, so here I stop.

#### d) Apply text normalization and second-order phrase model to text of complete reviews

Now what? I now have these useful models and lots of sentences, but I was originally looking at full reviews. Therefore, I can run the original texts through these models (and even remove imformation-poor stop-words to boot) and be left with a semantically-rich equivalent to our original text, ignoring most of grammar and focusing on the words themselves.

In [ ]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                       'trigram_tranformed_reviews_all.txt')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        for parsed_review in nlp.pipe(line_review(review_text_filepath),
                                    batch_size=10000, n_threads=4):
            
            # lemmatize, and remove punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review if not punct_space(token)]
            
            # apply the first- and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove stopwords, using SpaCy's nifty premade list
            trigram_review = [term for term in trigram_review if term not in spacy.en.STOP_WORDS]
            
            # finally, output this to the new file, review by review as line by line
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

This took a little more than 3 hours and 47 minutes to complete on my computer.

Thus, to compare the text from before and after NLP transformations:

In [ ]:
print(u'Original:\n')
for review in it.islice(line_review(review_text_filepath), 11, 12):
    print(review)
    
print(u'----\n')
print(u'Transformed:\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

It's obvious through a comparison of both of these that while a lot of content has been removed from the original text, the meat-and-potatoes remains. This information is far more useful for the semantic topic modelling that'll be applied next.

### Step 4: Visualizing topic models with pyLDAvis

*Topic Modeling* is a set of techniques where given a large corpus, inferences can be made between words such that many clusters of words appearing in similar contexts, or 'topics', can be determined and extracted. These topics can then be compared to the contexts found in specific documents, such that each document can be classified according to these contexts. In this sense, the computer is able to understand semantic connections between words without actually understanding the words themselves.

Here, [Latent Dirichlet Allocation](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf), or [LDA](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation), is to be used. This approach was designed in 2003 and proves highly successful and easy to implement in Python (though there are of course other topic modeling techniques that too could be used as well).

In LDA, documents are first analyzed not as massive strings, but merely as bags-of-words. A lexicon consisting of all the vocabulary found across all documents is created, then each document is described as a sparse matrix with the counts of each token found in each document. A massive and sparse *document-term matrix* results, and it is from here that analysis can be done.

The *Latent* in LDA can be seen as synonymous to *hidden*. There are a number of hidden topics that the documents can be described by, and the total number of topics is determined by human inference. In the examples below, 50 topics are found, though more or less would yield denser but less accurate topics or sparser but more comprehensive topics, respectively.

The [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) in LDA is a type of distribution discovered by Gustav Lejeune Dirichlet; this distribution is assumed to be the probability distribution that the documents, topics, and tokens jointly follow. Again, there are alternative options that could be utilized, but LDAs work quite well for instances like these.

Finally, it is important to note that this is an instance of unsupervised learning. This means that the algorithm will discover these 50 topics on its own, with no previous knowledge of what each topic is. Because of this, I will have to label each topic myself in the end, as their true definitions will remain hidden to the computer and to me.

Luckily, gensim again has modules that can be used for data processing and for LDA analysis.

In [ ]:
import warnings
# Because of Windows Anaconda, a warning appears without this filter.
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import _pickle as pickle

Gensim's *Dictionary* class allows the full lexicon of the Yelp review corpus to be modeled. The trigram dictionary found in the previous section will be used in forming the dictionary.

In [ ]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                          'trigram_dict_all.dict')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    trigram_reviews = LineSentence(trigram_reviews_filepath)
    
    # iterate the reviews and build a dictionary of trigrams
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are too rare or common (so little information gain) from the dict
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    
    # reassign integer ids removed from the filtering to conserve space
    trigram_dictionary.compactify()
    
    trigram_dictionary.save(trigram_dictionary_filepath)

# load the finished dictionary back in after it's been built
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

This process only about 5 minutes on my computer. The LDA utilizes a bag-of-words model to represent all the words and their counts, and this is now saved as a sparse matrix with the dimensions of the number of reviews by the number of total vocab words.

In [ ]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                   'trigram_bow_corpus_all.mm')

In [ ]:
def trigram_bow_generator(filepath):
    """
    generator function that reads in a file and yields a bow representation from it
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    #generate the bow representation for the reviews and save as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                      trigram_bow_generator(trigram_reviews_filepath))

# load the finished corpus back in after it's been built
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

This took about 9.5 minutes on my computer. I pass into the LDA model the sparse matrix made above, the number of topics to be found (here 50), and the total lexicon.

In [ ]:
lda_model_filepath = os.path.join(intermediate_directory,
                                 'lda_model_all')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again.
    
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        lda = LdaMulticore(trigram_bow_corpus,
                          num_topics=50,
                          id2word=trigram_dictionary,
                          workers=1)
        
    lda.save(lda_model_filepath)

# load the finished model back in after it's been built
lda = LdaMulticore.load(lda_model_filepath)

This took a little over 2 hours to complete. With this now-finished LDA model, each topic's most influential words can be seen, and from this one can infer roughly what each topic represents.

In [ ]:
def explore_topic(topic_number, topn=25):
    """
    prints out a formatted list of the top terms of a given topic number
    """
    
    print(u'{:20} {}\n'.format(u'term', u'frequency'))
    
    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
explore_topic(topic_number=15)

Topic 15 contains words like *bbq*, *sauce*, *smoke*, and *brisket*, all of which relate to **BBQ**. A similar process of inference has been repeated for each of the topics, and these have been listed below.

In [ ]:
topic_names = {0: u'interior',
              1: u'oriental',
              2: u'breakfast',
              3: u'service',
              4: u'arrival',
              5: u'cost',
              6: u'nightlife',
              7: u'alternative',
              8: u'hip',
              9: u'bar',
              10: u'japanese',
              11: u'pizza',
              12: u'bistro',
              13: u'steak',
              14: u'overnight',
              15: u'bbq',
              16: u'family',
              17: u'awesomesauce',
              18: u'dessert',
              19: u'selections',
              20: u'mexican',
              21: u'indian',
              22: u'discount',
              23: u'french',
              24: u'lunch',
              25: u'thai',
              26: u'location',
              27: u'parking/cajun/utensils',
              28: u'terribad',
              29: u'positive',
              30: u'buffet',
              31: u'tex-mex',
              32: u'cajun',
              33: u'burgers',
              34: u'ramen',
              35: u'fancy',
              36: u'alcohol/south',
              37: u'atmosphere',
              38: u'crowdedness',
              39: u'ice cream/delivery',
              40: u'french',
              41: u'management',
              42: u'experience',
              43: u'quality',
              44: u'letters/numbers',
              45: u'seafood',
              46: u'upper-class',
              47: u'celebration',
              48: u'breakfast diner',
              49: u'awesomesauce'}

In [ ]:
topic_names_filepath = os.path.join(intermediate_directory,
                                   'topic_names.pkl')

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

I've appended all of these topic names to the original file. Although not all topics are easy to infer, each can be roughly associated with some distinct aspect of restaurants and cuisine.

However, being able to visually inspect how each topic relates to one another in a graphical format would help to better understand just how each topic relates to one another. Luckily, pyLDAvis allows one to easily do such.

In [ ]:
LDAvis_data_filepath = os.path.join(intermediate_directory,
                                   'ldavis_prepared')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again
    
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda,
                                              trigram_bow_corpus,
                                              trigram_dictionary)
    
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the finished pyLDAvis data back in after it's been built       
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

With pyLDAvis.display(pyLDAvisModel), a visualization can be displayed directly in Jupyter.

In [ ]:
pyLDAvis.display(LDAvis_prepared)

...

In [ ]:
def get_sample_review(review_number):
    """
    retrieve a particular review index from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_text_filepath),
                         review_number,
                         review_number + 1))[0]

In [ ]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    take the original text of a review and
    (1) parse it using SpaCy
    (2) pre-process the text
    (3) create a bow representation
    (4) create an LDA representation
    (5) print a sorted list of the top topics from the LDA representation
    """
    
    # (1) parse the text using SpaCy
    parsed_review = nlp(review_text)
    
    # (2) pre-process the text
    #  - lemmatize and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                     if not punct_space(token)]
    
    #  - apply the first- and second-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    #  - remove stopwords
    trigram_review = [term for term in trigram_review
                     if not term in spacy.en.STOP_WORDS]
    
    # (3) create a bow representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # (4) create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly rated topics first
    review_lda = sorted(review_lda, key=(lambda topic_number_freq: -topic_number_freq[1]))
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
        
        # (5) print a sorted list of the top topics from the LDA representation
        print('{:25} {}'.format(topic_names[topic_number],
                               round(freq, 3)))

In [ ]:
sample_review = get_sample_review(50)
print(sample_review)

In [ ]:
lda_description(sample_review)

...

In [ ]:
sample_review = get_sample_review(100)
print(sample_review)

In [ ]:
lda_description(sample_review)

...

### Step 5: Word vector models with word2vec

In [ ]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

...

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again
    
    # initiate the model and perform training's first epoch
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                       min_count=20, sg=1, workers=2)
    
    food2vec.save(word2vec_filepath)
    
    # repeat another 11 epochs
    for i in range(1,12):
        food2vec.train(trigram_sentences,
                      total_examples=food2vec.corpus_count,
                      epochs=food2vec.iter)
        food2vec.save(word2vec_filepath)
        
# load the finished model back in after it's been built
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print(u'{} training epochs so far.'.format(food2vec.train_count))

This took over 15 hours total on my computer.

In [ ]:
print(u'{:,} terms in the food2vec vocabulary.'.format(len(food2vec.wv.vocab)))

...

In [ ]:
# build a list of the terms, integer indices, and term counts from food2vec's vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                for term, voc in food2vec.wv.vocab.items()]

# sort by term counts to order most frequent terms first
ordered_vocab = sorted(ordered_vocab, key=(lambda term_index_count: -term_index_count[2]))

# unzip the three into separate list
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a Pandas dataframe with the vectors as data and the terms as rows
word_vectors = pd.DataFrame(food2vec.wv.syn0norm[term_indices, :],
                           index=ordered_terms)

...

In [ ]:
def get_related_terms(token, topn=10):
    """
    Look up the topn most similar terms to the token and print them as  a list.
    """
    
    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):
        print(u'{:20} {}'.format(word, round(similarity, 3)))

...

In [ ]:
get_related_terms(u"mcdonald_'s")

...

In [ ]:
get_related_terms(u'happy_hour')

...

In [ ]:
get_related_terms(u'pasta', topn=20)

...

In [ ]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors of words in add and subtract,
    and print the topn most similar terms to the combined vector
    """
    
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

...

In [ ]:
word_algebra(add=[u'breakfast', u'lunch'])

...

In [ ]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

...

In [ ]:
word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])

...

In [ ]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

...

In [ ]:
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

...

In [ ]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

...

In [ ]:
word_algebra(add=[u"mcdonald_'s", u'fine_dining'])

...

In [ ]:
word_algebra(add=[u"denny_'s", u'fine_dining'])

...

In [ ]:
word_algebra(add=[u"applebee_'s", u'italian'])

...

In [ ]:
word_algebra(add=[u"applebee_'s", u'pancakes'])

...

In [ ]:
word_algebra(add=[u"applebee_'s", u'pizza'])

...

In [ ]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

...

### Step 6: Visualizing word2vec with t-SNE

In [ ]:
from sklearn.manifold import TSNE

...

In [ ]:
tsne_input = word_vectors.drop(spacy.en.STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

tsne_input.head()

...

In [ ]:
tsne_filepath = os.path.join(intermediate_directory,
                            u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                    u'tsne_vectors.npy')

In [ ]:
%%time
if False: # Once this is run once it does not need to be run again
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)
        
    pd.np.save(tsne_vectors_filepath, tsne_vectors)

    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

...

In [ ]:
tsne_vectors.head()

In [ ]:
tsne_vectors[u'word'] = tsne_vectors.index

...

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

In [ ]:
# add the dataframe as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create a plot and configure the external variables
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools = (u'pan, wheel_zoom, box_zoom',
                            u'box_select, resize, reset'),
                   active_scroll = u'wheel_zoom')

# add a hovel tool to display words on mouse-over
tsne_plot.add_tools(HoverTool(tooltips = u'@word'))

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                size=10, hover_line_color=u'black')

# configure the plot's visual elements
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# the plot is complete! Let's look at it!
show(tsne_plot)